In [1]:
# libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset


In [2]:
train = pd.read_csv('data/mnist_train.csv')
test = pd.read_csv('data/mnist_test.csv')

# Separate labels and features for training
train_labels = train.iloc[:, 0].values  # First column
train_pixels = train.iloc[:, 1:].values  # Remaining columns

# Separate labels and features for testing
test_labels = test.iloc[:, 0].values  # First column
test_pixels = test.iloc[:, 1:].values  # Remaining columns

# Normalize the pixel values to [0, 1]
train_pixels = train_pixels / 255.0
test_pixels = test_pixels / 255.0


# PyTorch Shenanigans
train_pixels_tensor = torch.tensor(train_pixels, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_pixels_tensor = torch.tensor(test_pixels, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)



In [4]:

# Assuming train_pixels, train_labels, test_pixels, test_labels are already preprocessed
# (single-channel, normalized to [0, 1], and in NumPy format)
# Convert to PyTorch tensors

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(train_pixels_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_pixels_tensor, test_labels_tensor)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Load pretrained ResNet18
resnet = models.resnet18(pretrained=True)

# Modify the first convolutional layer to accept single-channel (grayscale) images
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the final fully connected layer for MNIST classification (10 classes)
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

# Move the model to the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        # Reshape input to match ResNet input dimensions: [batch_size, 1, 28, 28]
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 28, 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


KeyboardInterrupt: 